In [1]:
#Importing necessary libraries
import hopsworks
import hsfs
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler  # Import StandardScaler from scikit-learn
import joblib

load_dotenv()

#Connecting to hopsworks
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

#Another connection to hopsworks
api_key = os.getenv('hopsworks_api')
connection = hsfs.connection()
fs = connection.get_feature_store()

2024-10-13 10:43:37,443 WARNING: DeprecationWarning: Type google._upb._message.MessageMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.

2024-10-13 10:43:37,444 WARNING: DeprecationWarning: Type google._upb._message.ScalarMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
2024-10-13 10:43:44,494 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
#Getting the feature view
feature_view = fs.get_feature_view(
    name='amd_stocks_fv',
    version=8
)

In [9]:
# Setting up train & test split dates
train_start = "2020-06-13"
train_end = "2023-12-31"

test_start = "2024-01-13"
test_end = "2024-10-13"

In [10]:
feature_view.create_train_test_split(
    train_start=train_start,
    train_end=train_end,
    test_start=test_start,
    test_end=test_end,
    data_format='csv',
    coalesce=True,
    statistics_config={'histogram': True, 'correlations': True}
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/564374/jobs/named/amd_stocks_fv_8_create_fv_td_13102024093206/executions
2024-10-13 11:33:36,220 WARNING: VersionWarning: Incremented version to `1`.



(1, <hsfs.core.job.Job at 0x1d3d159b7a0>)

In [11]:
#retrieving the training data from hopsworks
X_train, X_test, y_train, y_test = feature_view.get_train_test_split(1)

In [12]:
X_train

,date,f_1__open,f_2__high,f_3__low,f_5__volume,ticker
0,2023-12-12T00:00:00.000Z,132.250,139.8900,131.90,111677821.0,AMD
1,2023-11-28T00:00:00.000Z,121.980,122.5400,120.79,32047063.0,AMD
2,2021-05-24T00:00:00.000Z,77.260,78.0100,76.80,33194287.0,AMD
3,2021-11-22T00:00:00.000Z,157.140,161.8799,152.39,58674078.0,AMD
4,2023-01-30T00:00:00.000Z,74.230,74.6400,72.40,48105243.0,AMD
...,...,...,...,...,...,...
888,2020-10-06T00:00:00.000Z,86.210,87.2500,83.54,52725432.0,AMD
889,2021-04-20T00:00:00.000Z,80.820,81.1100,78.51,34977750.0,AMD
890,2021-08-12T00:00:00.000Z,107.390,108.4400,105.86,60625725.0,AMD
891,2023-12-07T00:00:00.000Z,120.785,128.6800,119.56,118579097.0,AMD


In [13]:
#Converting date into datetime
X_train['date'] = pd.to_datetime(X_train['date']).dt.date
X_test['date'] = pd.to_datetime(X_test['date']).dt.date
X_train['date'] = pd.to_datetime(X_train['date'])
X_test['date'] = pd.to_datetime(X_test['date'])

In [15]:
#inspecting the new format of the data
X_train.head(10)

,date,f_1__open,f_2__high,f_3__low,f_5__volume,ticker
0,2023-12-12,132.2500,139.8900,131.90,111677821.0,AMD
1,2023-11-28,121.9800,122.5400,120.79,32047063.0,AMD
2,2021-05-24,77.2600,78.0100,76.80,33194287.0,AMD
3,2021-11-22,157.1400,161.8799,152.39,58674078.0,AMD
4,2023-01-30,74.2300,74.6400,72.40,48105243.0,AMD
5,2022-01-31,107.9300,114.3100,106.90,97205994.0,AMD
6,2023-10-03,101.7963,103.2900,99.17,48116980.0,AMD
7,2020-12-24,91.8000,92.5100,91.31,16705909.0,AMD
8,2021-04-19,82.1300,83.1800,80.39,39115461.0,AMD
9,2022-06-28,85.7100,86.7300,80.43,95752533.0,AMD
